In [1]:
# Parameters
HPO_name = "greedy_HPO"
method_name = "EWC"
benchmark_name = "pMNIST"


HPO_name = "greedy_HPO"
method_name = "GroHess/Two layers"
benchmark_name = "pMNIST"

In [2]:
import sys
import os

sys.path.append("Methods/" + method_name)
path = os.path.dirname(os.path.abspath("__file__"))
data_path = path + "/Methods/" + method_name + "/data"

In [3]:
print(data_path)

/home/mathis/Master-Thesis-1/Workspace/Core work/Methods/EWC/data
